In [1]:
# Configuration section
import yfinance as yf
import pandas as pd
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns
import tkinter as tk
from tkinter import messagebox, ttk
import re
from datetime import datetime

# Configuration settings (if any)
# e.g., default values, constants

# Utility functions
def validate_date(date_text):
    """Validate date format."""
    return re.match(r'\d{4}-\d{2}-\d{2}', date_text) is not None

def perform_optimization(tickers, start_date, end_date):
    """Perform portfolio optimization."""
    # Fetch stock data
    data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    
    if data.empty:
        raise ValueError("No data fetched. Please check your ticker symbols and date range.")
    
    returns = data.pct_change().dropna()
    
    # Calculate expected returns and covariance matrix
    mu = expected_returns.mean_historical_return(data)
    S = risk_models.sample_cov(data)
    
    # Optimize portfolio
    ef = EfficientFrontier(mu, S)
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    ef.portfolio_performance(verbose=True)
    
    return cleaned_weights

# Main application logic
def optimize_portfolio():
    """Handle the portfolio optimization process."""
    tickers = tickers_entry.get().split(',')
    start_date = start_date_entry.get()
    end_date = end_date_entry.get()
    
    if not validate_date(start_date) or not validate_date(end_date):
        messagebox.showerror("Error", "Invalid date format. Please use YYYY-MM-DD.")
        return
    
    try:
        start_date_dt = datetime.strptime(start_date, '%Y-%m-%d')
        end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')
        
        result_text.set("Optimizing...")  # Indicate the process has started
        root.update_idletasks()  # Update the GUI
        
        weights = perform_optimization(tickers, start_date_dt, end_date_dt)
        result_text.set(f"Optimized Portfolio Weights: {weights}")
    except Exception as e:
        messagebox.showerror("Error", str(e))

# Set up the GUI
root = tk.Tk()
root.title("Portfolio Optimizer")

style = ttk.Style(root)
style.theme_use('clam')  # Use a modern theme

tk.Label(root, text="Tickers (comma-separated):").grid(row=0, column=0)
tickers_entry = tk.Entry(root)
tickers_entry.grid(row=0, column=1)

tk.Label(root, text="Start Date (YYYY-MM-DD):").grid(row=1, column=0)
start_date_entry = tk.Entry(root)
start_date_entry.grid(row=1, column=1)

tk.Label(root, text="End Date (YYYY-MM-DD):").grid(row=2, column=0)
end_date_entry = tk.Entry(root)
end_date_entry.grid(row=2, column=1)

optimize_button = tk.Button(root, text="Optimize", command=optimize_portfolio)
optimize_button.grid(row=3, column=0, columnspan=2)

result_text = tk.StringVar()
tk.Label(root, textvariable=result_text, wraplength=400).grid(row=4, column=0, columnspan=2)

root.mainloop()


[*********************100%%**********************]  4 of 4 completed


Expected annual return: 39.2%
Annual volatility: 38.8%
Sharpe Ratio: 0.96


[*********************100%%**********************]  4 of 4 completed


Expected annual return: 12.3%
Annual volatility: 29.0%
Sharpe Ratio: 0.35
